In [1]:
!pip install image_classifiers
#!pip install keras==3.1.1 tensorflow==2.16.1
import numpy as np
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential, Input
from keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from keras.applications.resnet_v2 import preprocess_input
from keras.models import Model
from classification_models.keras import Classifiers
from keras.optimizers import SGD
from keras.activations import linear
from keras.utils import Progbar
from keras.backend import clear_session
from tensorflow.nn import softmax_cross_entropy_with_logits
import os

2024-04-17 19:46:49.545807: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 19:46:50.448191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 50
batch_size = 64
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 83.46 MiB/s 1s

2024-04-17 19:46:58.829324: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:86:00.0, compute capability: 7.5


In [5]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def train_val_split(train_dataset, validation_size):
    X_train, y_train = train_dataset
    indices = np.random.permutation(X_train.shape[0])
    train_idx, val_idx = indices[:train_size], indices[train_size:]
    return (X_train[train_idx,...], y_train[train_idx,...]), (X_train[val_idx,...], y_train[val_idx,...])

def load_cifar_train_val():
    train_dataset, _ = cifar100.load_data()
    
    train_dataset, validation_dataset = train_val_split(train_dataset, validation_size)
    
    validation_dataset = tf.data.Dataset.from_tensor_slices(validation_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet).cache().prefetch(tf.data.AUTOTUNE)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(train_size).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, validation_dataset

In [6]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

@tf.function
def ce(x, y_logits, temp):
    return softmax_cross_entropy_with_logits(x, y_logits / temp) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy_etu, accuracy_ens, accuracy_dis, val_accuracy_etu, val_accuracy_ens, val_accuracy_dis\n")
def append_csv_log(fichier, epoch, accuracy_etu, accuracy_ens, accuracy_dis, val_accuracy_etu, val_accuracy_ens, val_accuracy_dis):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy_etu:.2f},{accuracy_ens:.2f},{accuracy_dis:.2f},{val_accuracy_etu:.2f},{val_accuracy_ens:.2f},{val_accuracy_dis:.2f}\n")

def forward_backward_pass_impl(train_dataset_iter, etudiant_logit_model, alpha, temp, optim):
    X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
    with tf.GradientTape() as tape:
        etudiant_estim_logit = etudiant_logit_model(X_batch, training = True)
        perte = alpha * softmax_cross_entropy_with_logits(y_batch,etudiant_estim_logit) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp)
    grads = tape.gradient(perte, etudiant_logit_model.trainable_variables)
    optim.apply_gradients(zip(grads, etudiant_logit_model.trainable_variables))
    return compte_bons(etudiant_estim_logit,y_batch), compte_bons(etudiant_logit_model, enseignant_estim_softmax)

@tf.function
def val_accuracies(etudiant, enseignant, validation_dataset):
    val_bons_etu, val_bons_dis = 0.0, 0.0
    for X_batch, y_batch in validation_dataset:
        etu_prev = etudiant(X_batch, training = False)
        ens_prev = enseignant(X_batch, training = False)
        val_bons_etu += compte_bons(etu_prev, y_batch)
        val_bons_dis += compte_bons(etu_prev, ens_prev)
    nb_obs_val = tf.cast((validation_size//batch_size) * batch_size, tf.float32)
    val_accuracy_etu = val_bons_etu / nb_obs_val
    val_accuracy_dis = val_bons_dis / nb_obs_val
    return val_accuracy_etu, val_accuracy_dis

def distillateur_kl(etudiant, enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha):
    etudiant_logit_model = get_modele_logits(etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    optim = SGD(learning_rate=0.001)
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    val_accuracy_etu_max, val_accuracy_dis_max  = 0, 0
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    forward_backward_pass = tf.function(forward_backward_pass_impl)
    # La tf.function ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        n_batch = train_size//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc (etu, train)", "acc (dis, train)"])
        bons_epoque_etu, bons_epoque_dis = 0, 0
        for i in range(n_batch):
            bons_etu, bons_dis = forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, optim).numpy()
            bons_epoque_etu += bons_etu.numpy()
            bons_epoque_dis += bons_dis.numpy()
            n_observ = (i+1) * batch_size
            accuracy_etu, accuracy_dis = bons_epoque_etu / n_observ, bons_epoque_dis / n_observ
            barre_progression.update(i + 1, values = [("acc (etu, train)", accuracy_etu), ("acc (dis, train)", accuracy_dis)])
        val_accuracy_etu, val_accuracy_dis = val_accuracies(etudiant, enseignant, validation_dataset)
        if val_accuracy_etu > val_accuracy_etu_max:
            val_accuracy_etu_max = val_accuracy_etu
            etudiant.save(f"sauvegardes/{nom_modele}_checkpoint.keras")
        if val_accuracy_dis > val_accuracy_dis_max:
            val_accuracy_dis_max = val_accuracy_dis
        if epoch + 1 == 40:
            optim.learning_rate.assign(0.0001)
        if epoch + 1 in [10,20,50]:
            print(f"---> Epoque {epoch + 1:d} - Max val accuracy -> etu : {val_accuracy_etu_max:.4f} | dis : {val_accuracy_dis_max:.4f}")
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy_etu, accuracy_ens, accuracy_dis, val_accuracy_etu, val_accuracy_ens, val_accuracy_dis)
        print(f"Accuracy (etu, val) : {val_accuracy_etu:.4f} | Accuracy (ens, val) : {val_accuracy_ens:.4f} | Accuracy (dis, val) : {val_accuracy_dis:.4f}")

In [7]:
def distillation_resnet18(temp, alpha):
    clear_session()
    train_dataset, validation_dataset = load_cifar_train_val()
    modele = new_modele_resnet()
    nom_modele =  f"model_etudiant_t{temp:d}_a{int(alpha*100):d}"
    distillateur_kl(modele, model_enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha)
    wd = os.getcwd()
    os.system(f"cp {wd}/sauvegardes/{nom_modele}_checkpoint.keras {wd}/sauvegardes/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

In [8]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [9]:
distillation_resnet18(1,1) # témoin

C'est parti pour la distillation !

Époque 1 / 50


2024-04-17 19:47:31.721100: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 456s 559ms/step - acc: 0.1007


I0000 00:00:1713383733.924931    9905 service.cc:145] XLA service 0x7f5794007ea0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713383733.936729    9905 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-17 19:55:34.410305: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1713383740.671648    9905 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Accuracy (train) : 0.1007 | Accuracy (val) : 0.0866
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 381s 543ms/step - acc: 0.2765
Accuracy (train) : 0.2765 | Accuracy (val) : 0.3430
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 403s 573ms/step - acc: 0.3794
Accuracy (train) : 0.3794 | Accuracy (val) : 0.4098
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 403s 573ms/step - acc: 0.4428
Accuracy (train) : 0.4428 | Accuracy (val) : 0.4780
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 406s 578ms/step - acc: 0.4847
Accuracy (train) : 0.4847 | Accuracy (val) : 0.5108
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 420s 597ms/step - acc: 0.5140
Accuracy (train) : 0.5140 | Accuracy (val) : 0.5478
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 410s 584ms/step - acc: 0.5393
Accuracy (train) : 0.5393 | Accuracy (val) : 0.5512
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 408s 581ms/step - acc: 0.5563
Accuracy (train) : 0.5563 | Accuracy (val) : 0.5752
Époque 9 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 608ms/step - acc: 0.5763


In [10]:
distillation_resnet18(1,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 445s 601ms/step - acc: 0.0981
Accuracy (train) : 0.0981 | Accuracy (val) : 0.0760
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 529ms/step - acc: 0.2729
Accuracy (train) : 0.2729 | Accuracy (val) : 0.3426
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 410s 583ms/step - acc: 0.3761
Accuracy (train) : 0.3761 | Accuracy (val) : 0.4168
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 420s 597ms/step - acc: 0.4349
Accuracy (train) : 0.4349 | Accuracy (val) : 0.4480
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 587ms/step - acc: 0.4790
Accuracy (train) : 0.4790 | Accuracy (val) : 0.4878
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 609ms/step - acc: 0.5092
Accuracy (train) : 0.5092 | Accuracy (val) : 0.5100
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 403s 574ms/step - acc: 0.5347
Accuracy (train) : 0.5347 | Accuracy (val) : 0.5570
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 600ms/step - acc: 0.5554
Accuracy (train)

2024-04-18 07:35:36.105474: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_zoom_1/add/_36}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-18 07:35:36.105557: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 3915092245815073522
2024-04-18 07:35:36.105574: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 13563200441571799962
2024-04-18 07:35:36.105585: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 17464713564312286712
2024-04-18 07:35:36.105605: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 16345586080016562991
2024-04-18 07:35:36.105617: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 165871482104

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a0.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a0.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 110.68 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a0_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a0_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 6.68 KiB/s


In [11]:
distillation_resnet18(3,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 446s 604ms/step - acc: 0.0831
Accuracy (train) : 0.0831 | Accuracy (val) : 0.1068
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 367s 522ms/step - acc: 0.2298
Accuracy (train) : 0.2298 | Accuracy (val) : 0.2844
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 441s 628ms/step - acc: 0.3401
Accuracy (train) : 0.3401 | Accuracy (val) : 0.3782
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 399s 568ms/step - acc: 0.4111
Accuracy (train) : 0.4111 | Accuracy (val) : 0.4626
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 410s 583ms/step - acc: 0.4641
Accuracy (train) : 0.4641 | Accuracy (val) : 0.4932
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 586ms/step - acc: 0.4976
Accuracy (train) : 0.4976 | Accuracy (val) : 0.5166
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 420s 598ms/step - acc: 0.5241
Accuracy (train) : 0.5241 | Accuracy (val) : 0.5490
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 400s 570ms/step - acc: 0.5470
Accuracy (train)

2024-04-18 13:28:12.808333: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_rotation_1/add/_32}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-18 13:28:12.809261: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 13563200441571799962
2024-04-18 13:28:12.809344: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 17464713564312286712
2024-04-18 13:28:12.809381: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 7957436170802050770
2024-04-18 13:28:12.809412: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 3915092245815073522
2024-04-18 13:28:12.809469: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 394339738

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a0.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a0.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 137.77 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a0_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a0_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 4.25 KiB/s


In [12]:
distillation_resnet18(8,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 452s 610ms/step - acc: 0.0503
Accuracy (train) : 0.0503 | Accuracy (val) : 0.0598
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 383s 544ms/step - acc: 0.1535
Accuracy (train) : 0.1535 | Accuracy (val) : 0.2364
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 610ms/step - acc: 0.2522
Accuracy (train) : 0.2522 | Accuracy (val) : 0.3062
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 415s 591ms/step - acc: 0.3272
Accuracy (train) : 0.3272 | Accuracy (val) : 0.3518
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 589ms/step - acc: 0.3839
Accuracy (train) : 0.3839 | Accuracy (val) : 0.3928
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 407s 579ms/step - acc: 0.4331
Accuracy (train) : 0.4331 | Accuracy (val) : 0.4212
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.4640
Accuracy (train) : 0.4640 | Accuracy (val) : 0.4506
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 410s 583ms/step - acc: 0.4944
Accuracy (train)

2024-04-18 19:27:33.599993: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/ReadVariableOp_1/_18}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-18 19:27:33.693435: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_translation_1/ReadVariableOp_1/_18}}]]
	 [[sequential_1_1/random_zoom_1/add/_36]] [type.googleapis.com/tensorflow.DerivedStatus='']


`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a0.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a0.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 59.58 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a0_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a0_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 4.29 KiB/s


In [13]:
distillation_resnet18(1,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 438s 589ms/step - acc: 0.1022
Accuracy (train) : 0.1022 | Accuracy (val) : 0.1008
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 366s 521ms/step - acc: 0.2814
Accuracy (train) : 0.2814 | Accuracy (val) : 0.3338
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 402s 572ms/step - acc: 0.3871
Accuracy (train) : 0.3871 | Accuracy (val) : 0.3982
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 399s 568ms/step - acc: 0.4504
Accuracy (train) : 0.4504 | Accuracy (val) : 0.4370
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 401s 571ms/step - acc: 0.4907
Accuracy (train) : 0.4907 | Accuracy (val) : 0.4946
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 407s 578ms/step - acc: 0.5189
Accuracy (train) : 0.5189 | Accuracy (val) : 0.5024
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 413s 588ms/step - acc: 0.5489
Accuracy (train) : 0.5489 | Accuracy (val) : 0.5166
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 410s 583ms/step - acc: 0.5660
Accuracy (train)

2024-04-19 01:23:45.771200: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 01:23:45.771866: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[sequential_1_1/random_translation_1/add_1/_42]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 01:23:45.774520: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 18100897507316354539
2024-04-19 01:23:45.774657: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-1

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 127.53 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a10_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 7.36 KiB/s


In [14]:
distillation_resnet18(3,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 450s 606ms/step - acc: 0.0752
Accuracy (train) : 0.0752 | Accuracy (val) : 0.0634
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 529ms/step - acc: 0.2220
Accuracy (train) : 0.2220 | Accuracy (val) : 0.2926
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 603ms/step - acc: 0.3289
Accuracy (train) : 0.3289 | Accuracy (val) : 0.3948
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 611ms/step - acc: 0.4000
Accuracy (train) : 0.4000 | Accuracy (val) : 0.4534
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 587ms/step - acc: 0.4527
Accuracy (train) : 0.4527 | Accuracy (val) : 0.5122
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 613ms/step - acc: 0.4946
Accuracy (train) : 0.4946 | Accuracy (val) : 0.5288
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 422s 601ms/step - acc: 0.5186
Accuracy (train) : 0.5186 | Accuracy (val) : 0.5474
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.5429
Accuracy (train)

2024-04-19 07:19:45.923830: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 07:19:45.924628: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_44]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 07:19:45.957040: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/ReadVariableOp/_2}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 07:19:45.959162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cance

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 77.95 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a10_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 4.90 KiB/s


In [15]:
distillation_resnet18(8,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 445s 602ms/step - acc: 0.0512
Accuracy (train) : 0.0512 | Accuracy (val) : 0.0468
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 530ms/step - acc: 0.1488
Accuracy (train) : 0.1488 | Accuracy (val) : 0.1942
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 407s 580ms/step - acc: 0.2433
Accuracy (train) : 0.2433 | Accuracy (val) : 0.2690
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 406s 578ms/step - acc: 0.3169
Accuracy (train) : 0.3169 | Accuracy (val) : 0.3510
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 408s 581ms/step - acc: 0.3749
Accuracy (train) : 0.3749 | Accuracy (val) : 0.4072
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 588ms/step - acc: 0.4201
Accuracy (train) : 0.4201 | Accuracy (val) : 0.4420
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 406s 578ms/step - acc: 0.4577
Accuracy (train) : 0.4577 | Accuracy (val) : 0.4796
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.4858
Accuracy (train)

2024-04-19 13:13:22.505495: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_rotation_1/add/_32}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-19 13:13:22.505678: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 13563200441571799962
2024-04-19 13:13:22.506692: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 7957436170802050770
2024-04-19 13:13:22.506732: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 3915092245815073522
2024-04-19 13:13:22.506782: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3943397384076495932
2024-04-19 13:13:22.506819: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvA

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a10.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a10.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 129.11 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a10_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a10_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 8.69 KiB/s


In [16]:
distillation_resnet18(1,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 464s 627ms/step - acc: 0.0998
Accuracy (train) : 0.0998 | Accuracy (val) : 0.1180
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 377s 536ms/step - acc: 0.2750
Accuracy (train) : 0.2750 | Accuracy (val) : 0.3294
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 505s 719ms/step - acc: 0.3808
Accuracy (train) : 0.3808 | Accuracy (val) : 0.4074
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 426s 606ms/step - acc: 0.4406
Accuracy (train) : 0.4406 | Accuracy (val) : 0.4458
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 440s 627ms/step - acc: 0.4798
Accuracy (train) : 0.4798 | Accuracy (val) : 0.4768
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 604ms/step - acc: 0.5189
Accuracy (train) : 0.5189 | Accuracy (val) : 0.5026
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 434s 618ms/step - acc: 0.5391
Accuracy (train) : 0.5391 | Accuracy (val) : 0.5186
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 433s 616ms/step - acc: 0.5579
Accuracy (train)

In [17]:
distillation_resnet18(3,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 461s 622ms/step - acc: 0.0803
Accuracy (train) : 0.0803 | Accuracy (val) : 0.0960
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 359s 511ms/step - acc: 0.2264
Accuracy (train) : 0.2264 | Accuracy (val) : 0.2828
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 607ms/step - acc: 0.3341
Accuracy (train) : 0.3341 | Accuracy (val) : 0.3608
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 604ms/step - acc: 0.4023
Accuracy (train) : 0.4023 | Accuracy (val) : 0.4150
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 589ms/step - acc: 0.4533
Accuracy (train) : 0.4533 | Accuracy (val) : 0.4608
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 611ms/step - acc: 0.4958
Accuracy (train) : 0.4958 | Accuracy (val) : 0.4914
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 430s 612ms/step - acc: 0.5210
Accuracy (train) : 0.5210 | Accuracy (val) : 0.5090
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 431s 613ms/step - acc: 0.5429
Accuracy (train)

2024-04-20 01:23:46.468244: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 01:23:46.472184: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3554000251992991798
2024-04-20 01:23:46.472307: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14771005561559515231
2024-04-20 01:23:46.472354: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]]
	 [[sequential_1_1/random_translation_1/add/_34]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 01:23:46.474228: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key ha

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a25.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a25.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 129.31 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a25_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a25_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 7.68 KiB/s


In [18]:
distillation_resnet18(8,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 455s 613ms/step - acc: 0.0517
Accuracy (train) : 0.0517 | Accuracy (val) : 0.0526
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 532ms/step - acc: 0.1560
Accuracy (train) : 0.1560 | Accuracy (val) : 0.2506
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 409s 582ms/step - acc: 0.2420
Accuracy (train) : 0.2420 | Accuracy (val) : 0.3250
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 600ms/step - acc: 0.3184
Accuracy (train) : 0.3184 | Accuracy (val) : 0.3846
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 588ms/step - acc: 0.3776
Accuracy (train) : 0.3776 | Accuracy (val) : 0.4198
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.4219
Accuracy (train) : 0.4219 | Accuracy (val) : 0.4580
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 413s 588ms/step - acc: 0.4569
Accuracy (train) : 0.4569 | Accuracy (val) : 0.4804
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 587ms/step - acc: 0.4875
Accuracy (train)

2024-04-20 07:19:44.403530: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/ReadVariableOp/_2}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 07:19:44.403856: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13563200441571799962
2024-04-20 07:19:44.406970: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17464713564312286712
2024-04-20 07:19:44.407274: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7957436170802050770
2024-04-20 07:19:44.407344: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3554000251992991798
2024-04-20 07:19:44.407565: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key has

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a25.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a25.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 107.99 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t8_a25_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t8_a25_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 5.09 KiB/s


In [19]:
distillation_resnet18(1,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 457s 613ms/step - acc: 0.1048
Accuracy (train) : 0.1048 | Accuracy (val) : 0.1112
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 543ms/step - acc: 0.2744
Accuracy (train) : 0.2744 | Accuracy (val) : 0.3352
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.3858
Accuracy (train) : 0.3858 | Accuracy (val) : 0.4238
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 610ms/step - acc: 0.4453
Accuracy (train) : 0.4453 | Accuracy (val) : 0.4518
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 603ms/step - acc: 0.4850
Accuracy (train) : 0.4850 | Accuracy (val) : 0.4726
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 428s 609ms/step - acc: 0.5186
Accuracy (train) : 0.5186 | Accuracy (val) : 0.5124
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 415s 591ms/step - acc: 0.5418
Accuracy (train) : 0.5418 | Accuracy (val) : 0.5382
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 429s 610ms/step - acc: 0.5601
Accuracy (train)

2024-04-20 13:14:06.002651: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node GroupCrossDeviceControlEdges_0/NoOp/_45}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 13:14:06.002843: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/add/_26}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 13:14:06.002878: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/add/_26}}]]
	 [[GroupCrossDeviceControlEdges_0/NoOp/_45]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 13:14:06.002929: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5308320966583080461
2024-04-20

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a50.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a50.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 125.84 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t1_a50_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t1_a50_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 7.04 KiB/s


In [20]:
distillation_resnet18(3,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 446s 601ms/step - acc: 0.0816
Accuracy (train) : 0.0816 | Accuracy (val) : 0.0858
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 364s 519ms/step - acc: 0.2290
Accuracy (train) : 0.2290 | Accuracy (val) : 0.3090
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.3391
Accuracy (train) : 0.3391 | Accuracy (val) : 0.3862
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 589ms/step - acc: 0.4167
Accuracy (train) : 0.4167 | Accuracy (val) : 0.4578
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 595ms/step - acc: 0.4620
Accuracy (train) : 0.4620 | Accuracy (val) : 0.5012
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 408s 580ms/step - acc: 0.4978
Accuracy (train) : 0.4978 | Accuracy (val) : 0.5016
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 411s 585ms/step - acc: 0.5271
Accuracy (train) : 0.5271 | Accuracy (val) : 0.5396
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 411s 584ms/step - acc: 0.5443
Accuracy (train)

2024-04-20 19:06:31.761117: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/add/_26}}]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 19:06:31.764064: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: CANCELLED: RecvAsync is cancelled.
	 [[{{node sequential_1_1/random_brightness_1/add/_26}}]]
	 [[sequential_1_1/random_translation_1/ReadVariableOp_3/_40]] [type.googleapis.com/tensorflow.DerivedStatus='']
2024-04-20 19:06:31.768548: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 14771005561559515231
2024-04-20 19:06:31.768647: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5308320966583080461
2024-04-20 19:06:31.768684: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv ite

`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a50.keras` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a50.keras`
Total: 43.54 MiB, Transferred: 43.54 MiB, Speed: 125.74 MiB/s
`/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant_t3_a50_logs.csv` -> `s3/afeldmann/projet_cnam/model_etudiant_t3_a50_logs.csv`
Total: 669 B, Transferred: 669 B, Speed: 6.25 KiB/s


In [21]:
distillation_resnet18(8,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 450s 606ms/step - acc: 0.0507
Accuracy (train) : 0.0507 | Accuracy (val) : 0.0584
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 543ms/step - acc: 0.1490
Accuracy (train) : 0.1490 | Accuracy (val) : 0.2344
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 421s 599ms/step - acc: 0.2448
Accuracy (train) : 0.2448 | Accuracy (val) : 0.3242
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 424s 604ms/step - acc: 0.3248
Accuracy (train) : 0.3248 | Accuracy (val) : 0.3828
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 419s 596ms/step - acc: 0.3807
Accuracy (train) : 0.3807 | Accuracy (val) : 0.4314
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 433s 616ms/step - acc: 0.4283
Accuracy (train) : 0.4283 | Accuracy (val) : 0.4582
Époque 7 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 602ms/step - acc: 0.4618
Accuracy (train) : 0.4618 | Accuracy (val) : 0.4806
Époque 8 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 420s 598ms/step - acc: 0.4945
Accuracy (train)